# Data Preparation and Advanced Model Evaluation

## Agenda

**Data preparation**

- Handling missing values
- Handling categorical features (review)

**Advanced model evaluation**

- ROC curves and AUC
- Bonus: ROC curve is only sensitive to rank order of predicted probabilities
- Cross-validation

## Part 1: Handling missing values

scikit-learn models expect that all values are **numeric** and **hold meaning**. Thus, missing values are not allowed by scikit-learn.

In [1]:
# read the Titanic data
import pandas as pd
url = '../../dataset/titanic.csv'
titanic = pd.read_csv(url, index_col='PassengerId')
titanic.shape

(891, 11)

In [2]:
titanic.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# check for missing values
titanic.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

One possible strategy is to **drop missing values**:

In [4]:
# drop rows with any missing values
titanic.dropna().shape

(183, 11)

In [5]:
# drop rows where Age is missing
titanic[titanic.Age.notnull()].shape

(714, 11)

In [6]:
titanic.Age.describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

Sometimes a better strategy is to **impute missing values**:

In [7]:
# mean Age
titanic.Age.mean()

29.69911764705882

In [8]:
# median Age
titanic.Age.median()

28.0

In [9]:
# most frequent Age
titanic.Age.mode()

0    24.0
dtype: float64

In [10]:
# create a dataframe with no missing values for ages
titanic_with_ages = titanic[titanic.Age.notnull()]
y = titanic_with_ages['Age']
X = titanic_with_ages.drop(['Age','Survived'], axis=1)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [12]:
# Feature Engineering Example with lambda function

X['ticket_contains_letters'] = X['Ticket'].apply(lambda x: x[0].isalpha())
X.head()

,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ticket_contains_letters
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,NaN,S,True
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C85,C,True
3,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,NaN,S,True
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,C123,S,False
5,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,NaN,S,False


## Create a model to predict the age

- Create new features
- Try different models
- Test differnt hyperparameters